# Stage Testing

In [2]:
import sys
import os

# Ruta al proyecto (ajusta si estás en otra carpeta)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

# 8-7-2025

## Basics

In [2]:
from app.services.stages.stage_logic import get_chat_stage_metadata
from app.core.aws_clients import get_dynamodb_client
from app.services.dynamodb_queries import get_latests_messages, response_to_conversation
from app.services.chatbot_engine import proccess_chat_turn

In [5]:
dynamodb= get_dynamodb_client()

In [6]:
user_id = "langchain-test-1"
conv_id = "langchain-test-1"
primary_key = f"USER#{user_id}#CONV#{conv_id}"

In [7]:
messages = get_latests_messages(dynamodb, primary_key, 10)

In [8]:
messages

{'Items': [{'metadata': {'M': {}},
   'SK': {'S': 'TIMESTAMP#2025-07-08T21:32:37.857423Z'},
   'message': {'S': 'Gracias por la información. ¿Tiene preferencia por una zona específica en Lima? ¿Qué tan importante es el acceso a transporte público o centros comerciales para usted? Con esta información, podremos filtrar opciones que mejor se ajusten a sus necesidades.'},
   'PK': {'S': 'USER#langchain-test-1#CONV#langchain-test-1'},
   'role': {'S': 'assistant'}},
  {'metadata': {'M': {}},
   'SK': {'S': 'TIMESTAMP#2025-07-08T21:32:36.924046Z'},
   'message': {'S': 'Pues prefiero un apartamento en algun condominio. No es necesario que tenga piscina. Mi presupuesto es de 2000 soles mensuales'},
   'PK': {'S': 'USER#langchain-test-1#CONV#langchain-test-1'},
   'role': {'S': 'user'}},
  {'metadata': {'M': {}},
   'SK': {'S': 'TIMESTAMP#2025-07-08T21:31:47.172809Z'},
   'message': {'S': 'Excelente criterio. ¿Prefiere un apartamento en un edificio con servicios adicionales (piscina, gimnasio)

In [9]:
br_conversation = response_to_conversation(messages)

In [10]:
br_conversation

[{'role': 'user', 'content': [{'text': 'Esto es una prueba con langchain'}]},
 {'role': 'user', 'content': [{'text': 'Esto es una prueba con langchain'}]},
 {'role': 'assistant',
  'content': [{'text': '¿Qué tipo de propiedad busca? Considere ubicación, tipo (apartamento, casa, etc.), tamaño, presupuesto y características específicas (cerca de transporte, escuelas, parques, etc.) para que podamos encontrar la mejor opción para usted.'}]},
 {'role': 'user',
  'content': [{'text': 'Estoy buscando un apartamento cerca a parques, en alguna zona segura de Lima'}]},
 {'role': 'assistant',
  'content': [{'text': 'Excelente criterio. ¿Prefiere un apartamento en un edificio con servicios adicionales (piscina, gimnasio)? ¿Cuál es su presupuesto aproximado? ¿Hay algún otro requisito específico que tenga en mente?'}]},
 {'role': 'user',
  'content': [{'text': 'Pues prefiero un apartamento en algun condominio. No es necesario que tenga piscina. Mi presupuesto es de 2000 soles mensuales'}]},
 {'role

In [5]:
user_id = "langchain-test-1"
conv_id = "langchain-test-1"
primary_key = f"USER#{user_id}#CONV#{conv_id}"

In [12]:
message = "Pues prefiero un apartamento en algun condominio. No es necesario que tenga piscina. Mi presupuesto es de 2000 soles mensuales"

In [13]:
response = proccess_chat_turn(user_id, conv_id, message)

In [14]:
response

'Gracias por la información. ¿Tiene preferencia por una zona específica en Lima? ¿Qué tan importante es el acceso a transporte público o centros comerciales para usted? Con esta información, podremos filtrar opciones que mejor se ajusten a sus necesidades.'

## structured output tests

In [2]:
from app.services.stages.stage1_extract import get_lead, message_history_build, has_minimium_data

In [3]:
invalid_conversation = [{'role': 'user', 'content': [{'text': 'Esto es otro texto de prueba, sin contexto'}]},
                        #{'role': 'user', 'content': [{'text': 'Quiero una propiedad en lima, con 3 baños, deseo alquilar. De preferencia con cercanía a centros comerciales'}]},
                        {'role': 'user', 'content': [{'text': 'También quisiera llevar mascotas'}]},
                        {'role': 'user', 'content': [{'text': 'Quisiera que fuera en san isidro, lima, peru'}]},
                        {'role': 'user', 'content': [{'text': 'Si quiero un departamento, y deseo alquilar'}]}
                        ]
response = get_lead(invalid_conversation)

In [4]:
response

PropertyLead(ubicacion='san isidro, lima, peru', tipo_propiedad='departamento', transaccion='alquiler', presupuesto=None, numero_dormitorios=None, numero_banos=None, metraje_minimo=None, zona_preferida='san isidro, lima, peru', amenidades=None, pet_friendly=True)

In [12]:
has_minimium_data(response)

True

## LangChain Handling Tests 

In [2]:
from app.services.stages.stage1_extract import handle
from app.services.stages.stage1_extract import get_missing_info, model_converse

In [9]:
conversation = [{'role': 'user', 'content': [{'text': 'Esto es otro texto de prueba, sin contexto'}]},
                        {'role': 'user', 'content': [{'text': 'Quiero una propiedad en lima, con 3 baños, deseo alquilar. De preferencia con cercanía a centros comerciales'}]},
                        {'role': 'user', 'content': [{'text': 'También quisiera llevar mascotas'}]},
                        {'role': 'user', 'content': [{'text': 'Quisiera que fuera en san isidro, lima, peru'}]},
                        #{'role': 'user', 'content': [{'text': 'Si quiero un departamento, y tengo un presupuesto de 2000 soles'}]}
                        ]

In [10]:
conversation

[{'role': 'user',
  'content': [{'text': 'Esto es otro texto de prueba, sin contexto'}]},
 {'role': 'user',
  'content': [{'text': 'Quiero una propiedad en lima, con 3 baños, deseo alquilar. De preferencia con cercanía a centros comerciales'}]},
 {'role': 'user', 'content': [{'text': 'También quisiera llevar mascotas'}]},
 {'role': 'user',
  'content': [{'text': 'Quisiera que fuera en san isidro, lima, peru'}]}]

In [11]:
response = handle(conversation)
response

{'conversation': [{'role': 'user',
   'content': [{'text': 'Esto es otro texto de prueba, sin contexto'}]},
  {'role': 'user',
   'content': [{'text': 'Quiero una propiedad en lima, con 3 baños, deseo alquilar. De preferencia con cercanía a centros comerciales'}]},
  {'role': 'user', 'content': [{'text': 'También quisiera llevar mascotas'}]},
  {'role': 'user',
   'content': [{'text': 'Quisiera que fuera en san isidro, lima, peru'}]}],
 'base_prompt': 'Simula ser un asesor inmobiliario que guía al usuario con preguntas para entender qué tipo de propiedad deseaNo respondas con recomendaciones aún. Sé breve pero cordial y amigable. (máx 50 palabras).Considera lo siguiente: Actualmente los datos faltantes son: {datos_faltantes}Los datos que podemos recolectar son los siguientes, tener en cuenta los campos requeridos, y los opcionales. Preguntar SOLAMENTE por los datos requeridos, puedes brindar información si consultan sobre campos opcionales: {data_info}Adicional: No respondas con palabr

In [7]:
response['formatted_conversation']

[('user', 'Esto es otro texto de prueba, sin contexto')]

In [6]:
model_response = model_converse(response['final_prompt'], response['formatted_conversation'])

ValidationException: An error occurred (ValidationException) when calling the Converse operation: A conversation must start with a user message. Try again with a conversation that starts with a user message.

In [ ]:
missing_list = get_missing_info(response['lead'])

In [7]:
missing_list

['tipo_propiedad',
 'presupuesto',
 'numero_dormitorios',
 'metraje_minimo',
 'amenidades']

In [8]:
", ".join(missing_list)

'tipo_propiedad, presupuesto, numero_dormitorios, metraje_minimo, amenidades'

## Data Context

In [2]:
from app.models.PropertyLead import PropertyLead

In [27]:
data_context = ""
for key,value in dict(PropertyLead.model_fields.items()).items():
    data_info = key +  ": " + value.description
    data_context += data_info + "\n"
data_context.strip("\n")

'ubicacion: Ubicación deseada como ciudad o distrito, etc\ntipo_propiedad: casa, departamento, cuarto, oficina, terreno, etc\ntransaccion: compra o alquiler\npresupuesto: Presupuesto aproximado en dolares o soles\nnumero_dormitorios: Cantidad mínima de dormitorios\nnumero_banos: Número mínimo de baños.\nmetraje_minimo: Área mínima en metros cuadrados\nzona_preferida: Alguna zona preferida de la ciudad\namenidades: Lista de amenidades deseadas.\npet_friendly: Si necesita que acepten mascotas.'

## Converse Troubleshooting

In [8]:
from app.services.stages.stage1_extract import get_langchain_bedrock_client

In [9]:
client = get_langchain_bedrock_client()

In [10]:
client()

C:\Users\Une\AppData\Local\Temp\ipykernel_23472\1206425969.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  client(response['formatted_conversation'])


AttributeError: 'tuple' object has no attribute 'content'

In [12]:
test_dict = {"test": "hola"}

In [16]:
if 'test2' in test_dict.keys():
    print("hi")
else:
    print("bye")

bye


In [17]:
LEAD_GENERATION_PARAMS = {"max_tokens": 250, "temperature": 0.6, "top_p" : 0.6}

In [19]:
max_tokens, temperature, top_p = LEAD_GENERATION_PARAMS

SyntaxError: invalid syntax (2337524538.py, line 1)

# 9-7-2025

## structured_output verification

In [2]:
from app.services.stages.stage1_extract import get_lead_with_prompt

In [3]:
prompt = "\n            A partir del siguiente historial de mensajes de usuario, extrae únicamente los datos explícitamente mencionados.\n            No hagas suposiciones. Si no se menciona algo, deja el campo como null.\n\n            Mensajes del usuario:\n             Estoy buscando un depa en San Isidro.\n\n            "
response = get_lead_with_prompt(prompt, include_raw=True)

In [4]:
response

{'raw': AIMessage(content=[{'type': 'tool_use', 'name': 'PropertyLead', 'input': {'metraje_minimo': None, 'numero_dormitorios': 1, 'zona_preferida': None, 'ubicacion': 'San Isidro', 'tipo_propiedad': 'departamento', 'presupuesto': None, 'transaccion': 'compra', 'pet_friendly': None, 'amenidades': None, 'numero_banos': None}, 'id': 'tooluse_0vYuYCoRRdSA7zMsESAH8Q'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '7b2d7b3d-31c0-4449-90c7-c9adb5a6d4a9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 09 Jul 2025 18:34:32 GMT', 'content-type': 'application/json', 'content-length': '486', 'connection': 'keep-alive', 'x-amzn-requestid': '7b2d7b3d-31c0-4449-90c7-c9adb5a6d4a9'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [640]}, 'model_name': 'amazon.nova-micro-v1:0'}, id='run--376ba32e-edb6-4b37-bd9a-59778ec2417c-0', tool_calls=[{'name': 'PropertyLead', 'args': {'metraje_minimo': None, 'numero_dormitorios': 1, 'zona_preferida': No

## stage 2 test

In [3]:
from app.services.stages.stage2_recommend import create_lead_description, handler

In [16]:
lead = {
    "ubicacion": "Lima, Perú, San Isidro",
    "tipo_propiedad": "departamento",
    "transaccion": "alquiler",
    "presupuesto": None,
    "numero_dormitorios": 2,
    "numero_banos": 2,
    "metraje_minimo": None,
    "amenidades": ["gimnasio"],
    "cercania": None,
    "pet_friendly": True
}

In [17]:
create_lead_description(lead)

"ubicacion: Lima, Perú, San Isidro, tipo_propiedad: departamento, transaccion: alquiler, numero_dormitorios: 2, numero_banos: 2, amenidades: ['gimnasio'], pet_friendly: True"

In [18]:
property_list = handler(lead)

In [20]:
str(property_list)

"[{'id': 'e2c976a5dc9248d2d5e63f131b4c3256', 'text': 'Departamento moderno en exclusivo San Isidro - Experience el lujo y confort de este departamento ubicado en el corazón de San Isidro, Lima. Con vistas panorámicas y proximidad a servicios públicos, esta es la oportunidad perfecta para tener un hogar único en una de las zonas más deseables de Lima. - casa - 471, Calle Teniente Paul de Beaudiez, Fondo de Vivienda Militar del Ejercito, San Isidro, Lima, Lima Metropolitana, Lima, 15076, Perú - alquiler', 'score': 0.0119888885}, {'id': '868a19bdd494fa59f393ff76978d4628', 'text': 'Apartamento moderno en Santa Isabel - Ofrecemos un espacioso apartamento en Santa Isabel, con vistas a la calle Mariscal Blas Cerdeña. Incluye ambiente común, 2 habitaciones, cocina integral y baño completo. - cuarto - 137, Calle Mariscal Blas Cerdeña, Santa Isabel, San Isidro, Lima, Lima Metropolitana, Lima, 15073, Perú - alquiler', 'score': 0.010497754}, {'id': 'c212f43aa045f3f0ca4d2b84b71c23ef', 'text': 'Cuar

## Get Chat Stage

In [3]:
from app.services.dynamodb_queries import get_all_messages, get_latests_messages, response_to_conversation, get_metadata
from app.core.aws_clients import get_dynamodb_client

In [4]:
client = get_dynamodb_client()

In [5]:
user_id="langchain-test-22"
conv_id=user_id
primary_key = "USER#"+user_id+"#CONV#"+conv_id

In [6]:
latest_messages = get_latests_messages(client, primary_key, limit=10)

In [7]:
latest_messages

{'Items': [{'metadata': {'M': {'additionalProp1': {'M': {}},
     'stage': {'S': 'recommend'},
     'lead': {'M': {'metraje_minimo': {'NULL': True},
       'numero_dormitorios': {'N': '1'},
       'ubicacion': {'S': 'Miraflores'},
       'tipo_propiedad': {'S': 'depa'},
       'presupuesto': {'NULL': True},
       'transaccion': {'S': 'alquiler'},
       'pet_friendly': {'NULL': True},
       'amenidades': {'NULL': True},
       'numero_banos': {'NULL': True},
       'cercania': {'NULL': True}}}}},
   'SK': {'S': 'TIMESTAMP#2025-07-09T22:43:20.236203Z'},
   'message': {'S': "[{'id': 'f28a046da59043df64ebab278aada7f5', 'text': 'Apartamento confortable en Miraflores, Lima - Área lujosa en el popular barrio de Miraflores. Apartamento equipado con todas las comodidades necesarias para su estancia, cerca del centro y de las principales atracciones turísticas. - departamento - Inkafarma, Avenida Mariscal La Mar, Santa Cruz, Miraflores, Lima, Lima Metropolitana, Lima, 15074, Perú - alquiler',

In [11]:
get_metadata(latest_messages)[-1]

{'additionalProp1': {},
 'stage': 'recommend',
 'lead': {'metraje_minimo': None,
  'numero_dormitorios': Decimal('1'),
  'ubicacion': 'Miraflores',
  'tipo_propiedad': 'depa',
  'presupuesto': None,
  'transaccion': 'alquiler',
  'pet_friendly': None,
  'amenidades': None,
  'numero_banos': None,
  'cercania': None}}